3.VAR SELECTION:

Vamos a preparar los dataframes, descargandolos de la base de datos sql "database9k.db" ademas de escoger las variables y preparar los dataframes para un FeatureEngineering óptimo. Por lo que haremos una primera tanteada de
modelos de clasificacion y escogeremos en funcion del F1 score, precisión , etc.


In [ ]:
#1) Creacion de Subgrupos de paises:
import pandas as pd
from sqlalchemy import create_engine, text
import os
import sqlite3
#1.1 Nos conectamos a la base de datos.
database_path = os.path.abspath("C:/GitHubRepos/ProyectoFinal/data/processed/Dataset9k.db")
print(f"Database path: {database_path}")
conn = create_engine(f'sqlite:///{database_path}')
# Execute the SQL query, ya hemos visto en el visualizador de sql que los datos que deberian ser numericos hay algunas columnas que son formato texto, 
# hacemos un query para transformarlos justo antes de almacenarlos en un dataframe

#DF inflacion
query = """
SELECT País, REPLACE(Inflación, ',', '.') AS Inflación_Float
FROM EuroInfl
"""
#Creamos un diccionario para almacenar la conversion
dtype_dict = {'Inflación_Float': float}
# Load the DataFrame with the dtype dictionary
df_inf = pd.read_sql_query(query, conn, dtype=dtype_dict)
df_inf.drop_duplicates(subset='País', keep='first', inplace=True)

#DF PPC
query = f"""
SELECT País, ROUND(PPC) AS PPC_INTEGER
FROM EuroPPC
"""
df_PPC = pd.read_sql_query(query, conn)
df_PPC.drop_duplicates(subset='País', keep='first', inplace=True)
print(df_PPC)

#DF SBMM
query = """
SELECT País, REPLACE(SBMM, ',', '.') AS SBMM_Float
FROM EuroSBMM
"""
dtype_dict = {'SBMM': float}
df_SBMM = pd.read_sql_query(query, conn)
df_SBMM.drop_duplicates(subset='País', keep='first', inplace=True)
df_SBMM['País'] = df_SBMM['País'].str.strip().str.lower()

conn.dispose()
print(df_SBMM)


In [ ]:
#No hay forma de cambiar la columna paises de la tabla SBMM, por lo que habra que realizar un diccionario de equivalencias para hacer coincidir la columna paises en los tres
#Dataframes de graficos, realizaremos posteriormente una clusterizacion para elegir 3 subgrupos de paises con economias similares y poder compararlos
#A nivel general con la Union Europea.
país_equivalencias = {
    'bulgaria :bg:': 'Bulgaria',
    'rumanía :ro:': 'Rumania',
    'hungría :hu:': 'Hungría',
    'croacia :hr:': 'Croacia',
    'polonia :pl:': 'Polonia',
    'letonia :lv:': 'Letonia',
    'eslovaquia :sk:': 'República Eslovaca',
    'grecia :gr:': 'Grecia',
    'lituania :lt:': 'Lituania',
    'estonia :ee:': 'Estonia',
    'portugal :pt:': 'Portugal',
    'república checa :cz:': 'República Checa',
    'malta :mt:': 'Malta',
    'chipre :cy:': 'Chipre',
    'eslovenia :si:': 'Eslovenia',
    'españa :es:': 'España',
    'italia :it:': 'Italia',
    'media ue :eu:': 'Unión Europea',
    'francia :fr:': 'Francia',
    'suecia :se:': 'Suecia',
    'finlandia :fi:': 'Finlandia',
    'austria :at:': 'Austria',
    'países bajos :nl:': 'Países Bajos',
    'bélgica :be:': 'Bélgica',
    'irlanda :ie:': 'Irlanda',
    'alemania :de:': 'Alemania',
    'dinamarca :dk:': 'Dinamarca',
    'luxemburgo :lu:': 'Luxemburgo',
}
df_SBMM['País'] = df_SBMM['País'].map(país_equivalencias)
print(df_SBMM)

In [ ]:
#Antes de la concatenacion de los dataframe hemos de hacer coincidir el tipo de datos, coincidir el index 

df_inf.reset_index(drop=True, inplace=True)
print(df_inf['País'].dtype)
print(df_inf['Inflación_Float'].dtype)
print(df_inf)

In [ ]:
#df_PPC dtypes
df_PPC.reset_index(drop=True, inplace=True)
print(df_PPC['País'].dtype)
print(df_PPC['PPC_INTEGER'].dtype)

In [ ]:
#df_SBMM
df_SBMM.reset_index(drop=True, inplace=True)
print(df_SBMM['País'].dtype)
print(df_SBMM['SBMM_Float'].dtype)
#Aqui tenemos el problema, hemos de pasar la columna SBMM_FLoat a float64
df_SBMM['SBMM_Float'] = df_SBMM['SBMM_Float'].astype('float64')
print(df_SBMM['SBMM_Float'].dtype)

In [ ]:
#Preparamos el dataframe concatenado filtrando por valores unicos de paises y sus valores numericos en cada uno de los dataframes.
# Encontrar países comunes
paises_comunes = set(df_inf['País']).intersection(set(df_PPC['País']), set(df_SBMM['País']))

# Filtrar DataFrames para países comunes y establecer 'País' como índice
df_i1 = df_inf[df_inf['País'].isin(paises_comunes)].set_index('País')
df_P1 = df_PPC[df_PPC['País'].isin(paises_comunes)].set_index('País')
df_S1 = df_SBMM[df_SBMM['País'].isin(paises_comunes)].set_index('País')
# Concatenar los DataFrames por columnas
df_IPS = pd.concat([df_i1, df_P1, df_S1], axis=1)
#Eliminamos variables usadas que no son inutiles.
df_IPS.to_csv('C:/GitHubRepos/ProyectoFinal/data/processed/EuEco.csv', index=True)
del df_i1, df_P1, df_S1, df_inf, df_SBMM, df_PPC, país_equivalencias, dtype_dict, paises_comunes, df_IPS
df1= pd.read_csv('C:\GitHubRepos\ProyectoFinal\data\processed\EuEco.csv')
print(df1)


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#1.5 Representacion Graficos Economicos.
# Crear una grupo de graficos lineales con 3 subplots en una fila
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 8))
# Graficar cada DataFrame en un subplot diferente
axes[0].plot(df1['País'], df1['Inflación_Float'])
axes[0].set_title('Inflacion Europa 2023')
axes[0].set_xlabel('Paises Euro')
axes[0].set_ylabel('Porcentaje')
axes[0].tick_params(axis='x', rotation=90)  

axes[1].plot(df1['País'], df1['SBMM_Float'], color='Green')
axes[1].set_title('Sueldo Bruto Mensual Medio')
axes[1].set_xlabel('Paises Euro')
axes[1].set_ylabel('Euros')
axes[1].set_ylim(ymin=0) 
axes[1].tick_params(axis='x', rotation=90)  

axes[2].plot(df1['País'], df1['PPC_INTEGER'],  color='Orange')
axes[2].set_title('PIB Per Capita Anual')
axes[2].set_xlabel('Paises Euro')
axes[2].set_ylabel('Miles de Euros')
axes[2].set_ylim(ymin=0)  
axes[2].tick_params(axis='x', rotation=90) 
plt.tight_layout()
plt.show()
df1.set_index('País', inplace=True)
for column in ['Inflación_Float', 'PPC_INTEGER', 'SBMM_Float']:
    sns.lineplot(data=df1, x=df1.index, y=column, label=column)
plt.title('Evolución de indicadores económicos por país')
plt.xlabel('País')
plt.ylabel('Valor')
plt.legend(title='Indicador')
plt.xticks(rotation=90)

plt.show()
del axes

In [ ]:
#Ahora vamos a crear 3 subgrupos de paises: 
#Los haremos automaticamente con KMeans, podemos predecir los grupos
#por las caracteristicas economicas.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import plotly.express as px

X = df1[['Inflación_Float', 'PPC_INTEGER', 'SBMM_Float']]
kmeans = KMeans(n_clusters=3, random_state=0).fit(X)
df1['cluster'] = kmeans.labels_
fig = px.scatter_3d(df1, x='Inflación_Float', y='PPC_INTEGER', z='SBMM_Float',
              color='cluster',
              opacity=0.7,
              color_continuous_scale='viridis',
              symbol='cluster',
              size_max=18)
fig.update_layout(
    title='Clustering de países en 3D',
    scene = dict(
        xaxis_title='Inflación',
        yaxis_title='PPC',
        zaxis_title='SBMM'
    ),
    legend=dict(
        title='Cluster',
        yanchor="top",
        xanchor="right"
    )
)

fig.show()

In [ ]:
#Ahora llamamos a los valores unicos del indice default del df1 que se agrupan en cada cluster para seleccionar ademas 
#del criterio matematico, razones geograficas y culturales.

for i in range(kmeans.n_clusters):
    cluster_i = df1[df1['cluster'] == i].index
    print(f"Países en el cluster {i}:")
    print(cluster_i.tolist())


In [ ]:
#Cargamos el df base del servidor sql.
import pandas as pd
import sqlite3
conn = sqlite3.connect('C:\GitHubRepos\ProyectoFinal\data\processed\Dataset9k.db')
query = "SELECT * FROM ZonaEuroESS"
df_encuestas = pd.read_sql_query(query, conn)
conn.close()

In [ ]:
#Factorizamos nuestra variable target ()
import json
import os
mapping = {'SocialComunista': 0, 'Socialista': 1, 'Demócrata(Centro)': 2, 'Conservador': 3, 'ExtremaDerecha': 4}
def agrupar_lrscale(valor):
    if 0 <= valor <= 2:
        return 'SocialComunista'
    elif 3 == valor <= 4:
        return 'Socialista'
    elif 5 == valor <= 6:
        return 'Demócrata(Centro)'
    elif 7 == valor <= 8:
        return 'Conservador'
    else:
        return 'ExtremaDerecha'


df_encuestas['ideo'] = df_encuestas['lrscale'].apply(agrupar_lrscale)





df_encuestas['ideo_fc'] = df_encuestas['ideo'].map(mapping)


ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/ideo_factors.json"


with open(ruta_archivo_json, 'w') as f:
    json.dump(mapping, f)
df_encuestas = df_encuestas.apply(lambda col: col.astype(int) if col.dtype == 'float64' else col)
print(df_encuestas.head())

In [ ]:
import pandas as pd

# Supongamos que df_encuestas y df1 ya están cargados

# Fusionar los dataframes
df_encuestas = pd.merge(df_encuestas, df1[['cluster']], left_on='cntry', right_index=True, how='left')

# Manejar valores faltantes
df_encuestas['cluster'] = df_encuestas['cluster'].fillna(3)
#CLUSTER 0 = PAISES RICOS. CLUSTER 1 = PAISES INFERIORES A LA MEDIA DE LA UE(CLUSTER 2), CLUSTER 2= ZONA MEDIA EUW(ITALIA, ESPAÑA, CLUSTER 3= PERIFERIA EUW(ISRAEL, TURQUIA, ETC.))
print(df_encuestas.head())

In [ ]:
#Ahora que tenemos etiquetadas las muestras por Clusters, zonas geograficas y grupos economicos. Podemos reducir las muestras para trabajar con ellas.
#1) Vamos a reducir el dataframe a 2mil muestras por cluster. Vamos a eliminar las muestras del Cluster NUmero 3 en nuestro nuevo dataframe ya que no nos interesa los paises perifericos a europa por diferencias culturales entre otros motivos.
import pandas as pd
df_clusters = {}
for cluster in df_encuestas['cluster'].unique():
    df_clusters[cluster] = df_encuestas[df_encuestas['cluster'] == cluster]

tamano_muestra = 2000

dfC0a2 = pd.concat([
    df_clusters[0].sample(n=tamano_muestra, random_state=42),
    df_clusters[1].sample(n=tamano_muestra, random_state=42),
    df_clusters[2].sample(n=tamano_muestra, random_state=42),
    df_clusters[3].sample(n=tamano_muestra, random_state=42)
])

print(dfC0a2 .head())  
print(dfC0a2 ['cluster'].value_counts())
dfC0a2 = dfC0a2.apply(lambda col: col.astype(int) if col.dtype == 'float64' else col)
dfc= dfC0a2.copy()

In [14]:
del X, df1, df_clusters, df_encuestas, dfC0a2['cntry'], dfC0a2['orig'], dfC0a2['ideo'], dfC0a2['lrscale'], mapping
#Eliminamos las columnas sin factorizar.

In [ ]:
'''#Eliminamos CSV y archivos innecesarios. Tenemos el raw en la database, si queremos trabajar de forma distina con ellos.
import os
def eliminar_archivo_csv(ruta_archivo):
  try:
    os.remove(ruta_archivo)
    print(f"El archivo CSV '{ruta_archivo}' ha sido eliminado.")
  except FileNotFoundError:
    print(f"El archivo CSV '{ruta_archivo}' no existe.")
  except Exception as e:
    print(f"Ocurrió un error al eliminar el archivo CSV: {e}")
ruta_del_archivo = "C:\GitHubRepos\ProyectoFinal\data\processed\dfencuestas.csv"  # Reemplaza con la ruta real de tu archivo
eliminar_archivo_csv(ruta_del_archivo)'''

In [ ]:
#Guardamos el dataframe de 6mil muestras antes de hacer test de score para seleccionar el dataframe para el Eda.
dfC0a2.to_csv('C:\GitHubRepos\ProyectoFinal\data\processed\df6k.csv', index=False)

In [ ]:
#Vamos probar un Random Forest con 3 dataframes con distintos numeros de variables, nos quedaremos con el que obtenga mas score de los tres.
#La variable ['ideo_fc'] es nuestra variable target.
#Modelo uno 11 variables(10 predictoras, 1 target), 16 vars, 21 vars.
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2 #(chi2 es el criterio matematico apropiado para vars categoricas factorizadas.)

def seleccionar_mejores_variables(df, k, target='ideo_fc'):
    X = df.drop(target, axis=1)
    y = df[target]
    selector = SelectKBest(chi2, k=k) 
    X_new = selector.fit_transform(X, y)
    mejores_features = X.columns[selector.get_support()]
    print(f"Mejores features ({k} features):", mejores_features)
    df_seleccionado = df[[target] + list(mejores_features)]
    return df_seleccionado
d11= seleccionar_mejores_variables(dfC0a2, 10)  # 10 predictoras + 1 objetivo = 11
d16 = seleccionar_mejores_variables(dfC0a2, 15)  # 15 predictoras + 1 objetivo = 16
d21 = seleccionar_mejores_variables(dfC0a2, 20)  # 20 predictoras + 1 objetivo = 21

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Supongamos que ya tienes los dataframes df_11, df_16 y df_21

# Función para entrenar y evaluar un modelo
def entrenar_y_evaluar(X_train, y_train, X_test, y_test):
    modelo = RandomForestClassifier(random_state=42)
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    print(classification_report(y_test, y_pred))
    return modelo  # Devuelve el modelo entrenado

# Entrenar y evaluar con df_11
X_11 = d11.drop('ideo_fc', axis=1)
y_11 = d11['ideo_fc']
X_train_11, X_test_11, y_train_11, y_test_11 = train_test_split(
    X_11, y_11, test_size=0.2, random_state=42
)
print("Resultados con df_11:")
modelo_11 = entrenar_y_evaluar(X_train_11, y_train_11, X_test_11, y_test_11)

# Entrenar y evaluar con df_16
X_16 = d16.drop('ideo_fc', axis=1)
y_16 = d16['ideo_fc']
X_train_16, X_test_16, y_train_16, y_test_16 = train_test_split(
    X_16, y_16, test_size=0.2, random_state=42
)
print("\nResultados con df_16:")
modelo_16 = entrenar_y_evaluar(X_train_16, y_train_16, X_test_16, y_test_16)

# Entrenar y evaluar con df_21
X_21 = d21.drop('ideo_fc', axis=1)
y_21 = d21['ideo_fc']
X_train_21, X_test_21, y_train_21, y_test_21 = train_test_split(
    X_21, y_21, test_size=0.2, random_state=42
)
print("\nResultados con df_21:")
modelo_21 = entrenar_y_evaluar(X_train_21, y_train_21, X_test_21, y_test_21)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Función para entrenar y evaluar un modelo (adaptada para regresión logística)
def entrenar_y_evaluar_logreg(X_train, y_train, X_test, y_test):
    modelo = LogisticRegression(random_state=42, max_iter=1000)  # Aumentamos max_iter si es necesario
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    print(classification_report(y_test, y_pred))
    return modelo

# Entrenar y evaluar con df_11_seleccionado
X_11 = d11.drop('ideo_fc', axis=1)
y_11 = d11['ideo_fc']
X_train_11, X_test_11, y_train_11, y_test_11 = train_test_split(
    X_11, y_11, test_size=0.2, random_state=42
)
print("Resultados con df_11_seleccionado (Regresión Logística):")
modelo_11_logreg = entrenar_y_evaluar_logreg(X_train_11, y_train_11, X_test_11, y_test_11)

# Entrenar y evaluar con df_16_seleccionado
X_16 = d16.drop('ideo_fc', axis=1)
y_16 = d16['ideo_fc']
X_train_16, X_test_16, y_train_16, y_test_16 = train_test_split(
    X_16, y_16, test_size=0.2, random_state=42
)
print("\nResultados con df_16_seleccionado (Regresión Logística):")
modelo_16_logreg = entrenar_y_evaluar_logreg(X_train_16, y_train_16, X_test_16, y_test_16)

# Entrenar y evaluar con df_21_seleccionado
X_21 = d21.drop('ideo_fc', axis=1)
y_21 = d21['ideo_fc']
X_train_21, X_test_21, y_train_21, y_test_21 = train_test_split(
    X_21, y_21, test_size=0.2, random_state=42
)
print("\nResultados con df_21_seleccionado (Regresión Logística):")
modelo_21_logreg = entrenar_y_evaluar_logreg(X_train_21, y_train_21, X_test_21, y_test_21)

In [ ]:
#Dado los bajos resultados obtenidos, aun siendo en default, voy a hacer un repaso de la naturaleza de las variables de nuestro dataframe, y factorizar escalas de intensidad 0 a 10.
#Probaremos nuestra target tanto factorizada como en escala.
print(dfC0a2.columns.tolist())

In [ ]:
#Bloque1(media): dfC0a2['netusoft', 'nwsptot', 'ppltrst', 'rdtot', 'tvtot']
#Vamos a unir todas las variables que hacen referencia al uso de medios de comunicaciones oficiales.
#Nuestro criterio no sera el tiempo sino el habito, si es nulo, ocasional o rutinario.
#['netusoft','nwsptot','rdtot','tvtot']
#Factorizamos nuestra variable target ()
mapping = {'Aleatoriamente': 0, 'Ocasionalmente': 1, 'Rutinariamente': 2}
def categorizar_frecuencia(valor):
    if 0 <= valor <= 2:
        return 'Aleatoriamente'
    elif valor == 3:
        return 'Ocasionalmente'
    elif valor >= 4:
        return 'Rutinariamente'
    else:
        return None 
variables = ['netusoft', 'nwsptot', 'rdtot', 'tvtot']
for variable in variables:
    # Categoriza la frecuencia
    dfc[variable] = dfc[variable].apply(categorizar_frecuencia)
    
    # Mapea a valores numéricos (factorización)
    dfc[f'{variable}_fc'] = dfc[variable].map(mapping)
    
    # Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
    ruta_archivo_json = f"C:/GitHubRepos/ProyectoFinal/data/processed/{variable}_factors.json"
    with open(ruta_archivo_json, 'w') as f:
        json.dump(mapping, f)
dfc = dfc.drop(variables, axis=1)
print(dfc.head())

In [ ]:
#Combinaremos las variables de media priorizando el valor de netusfot(0 si es rutinario se asignara al encuestado el valor factorizado
# de 'Alternativo', si esta var tiene un valor diferente a rutinario se asignara el valor Oficial si alguna de las otras variables
# tienee l valor de rutinario, si no se asignara el valor A ofimed = aleatorio.)
def combinar_medios(row):
    """Combina las variables de uso de medios según el criterio especificado."""

    if row['netusoft_fc'] == 2:  # netusoft rutinariamente (2)
        return 'Alternativo'
    elif row['nwsptot_fc'] == 2 or row['rdtot_fc'] == 2 or row['tvtot_fc'] == 2:
        return 'Oficial'
    elif row['nwsptot_fc'] == 1 or row['rdtot_fc'] == 1 or row['tvtot_fc'] == 1:
        return 'Ocasional'
    else:
        return 'Aleatorio'

dfc['OfiMed'] = dfc.apply(combinar_medios, axis=1)

# Mapea 'OfiMed' a valores numéricos (factorización)
mapping_ofimed = {'Aleatorio': 0, 'Ocasional': 1, 'Alternativo': 3, 'Oficial': 4}  # Ajusta el mapeo
dfc['OfiMed_fc'] = dfc['OfiMed'].map(mapping_ofimed)

print(dfc[['netusoft_fc', 'nwsptot_fc', 'rdtot_fc', 'tvtot_fc', 'OfiMed', 'OfiMed_fc']].head())

In [23]:
del dfc['netusoft_fc'],dfc['nwsptot_fc'],dfc['rdtot_fc'],dfc['tvtot_fc'],dfc['OfiMed']

In [ ]:
import pandas as pd
import json

mappingtrust = {'Desconfiado': 0, 'Esceptico': 1, 'Selectivo': 2, 'Confiado': 3, 'Oficialista': 4}

def categorizar_confianza(valor):
    if 0 <= valor <= 1:
        return 'Desconfiado'
    elif 2 <= valor <= 4:
        return 'Esceptico'
    elif 5 <= valor <= 6:
        return 'Selectivo'
    elif 7 <= valor <= 8:
        return 'Confiado'
    else:  # La condición correcta para valores 9 y 10
        return 'Oficialista'

dfc['ppltrst'] = dfc['ppltrst'].apply(categorizar_confianza)
dfc['ppltrst_fc'] = dfc['ppltrst'].map(mappingtrust)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/ppltrst_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappingtrust, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['ppltrst', 'ppltrst_fc']].head())


In [25]:
del dfc['ppltrst']

In [ ]:
#Vamos a factorizar todas las escalas de 0 a 10 del bloque 2 político.
#BLOQUE 2: df_rd['actrolga','bctprd','dclagr','dclaid','dclcrm','dclenv', 'dclmig', 'dclwlfr', 
# 'dmcntov', 'euftf', 'gincdif', 'ginveco', 'lawobey', 'lrscale', 'polintr','stfeco','stfedu',
# 'stfgov', 'stfhlth', 'stflife', 'trstplc', 'trstplt', 'trstun', 'trstsci', 'imbgeco']
#var 'dmcntov'
mappingdemo = {'Dictadura': 0, 'PocoDem': 1, 'Democratico': 2}

def demo(valor):
    if 0 <= valor <= 3:
        return 'Dictadura'
    elif 4 <= valor <= 7:
        return 'PocoDem'
    else: 
        return 'Democratico'

dfc['dmcntov'] = dfc['dmcntov'].apply(demo)
dfc['dmcntov_fc'] = dfc['dmcntov'].map(mappingdemo)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/dmcntov_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappingdemo, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['dmcntov', 'dmcntov_fc']].head())


In [ ]:
#BLOQUE 2: df_rd['actrolga','bctprd','dclagr','dclaid','dclcrm','dclenv', 'dclmig', 'dclwlfr', 
# 'dmcntov', 'euftf', 'gincdif', 'ginveco', 'lawobey', 'lrscale', 'polintr','stfeco','stfedu',
# 'stfgov', 'stfhlth', 'stflife', 'trstplc', 'trstplt', 'trstun', 'trstsci', 'imbgeco']

#var 'euftf' La unificacion de la zona euro ha ido demasiado lejos o necesita ir mas lejos.
mappingeu = {'DemasiadoLejos': 0, 'EstaBien': 1, 'NecesitaIrMasLejos': 2}
def demo(valor):
    if 0 <= valor <= 3:
        return 'DemasiadoLejos'
    elif 4 <= valor <= 7:
        return 'EstaBien'
    else: 
        return 'NecesitaIrMasLejos'

dfc['euftf'] = dfc['euftf'].apply(demo)
dfc['euftf_fc'] = dfc['euftf'].map(mappingeu)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/euftf_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappingeu, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['euftf', 'euftf_fc']].head())

In [ ]:
#BLOQUE 2: df_rd['actrolga','bctprd','dclagr','dclaid','dclcrm','dclenv', 'dclmig', 'dclwlfr', 
# 'dmcntov', 'euftf', 'gincdif', 'ginveco', 'lawobey', 'lrscale', 'polintr','stfeco','stfedu',
# 'stfgov', 'stfhlth', 'stflife', 'trstplc', 'trstplt', 'trstun', 'trstsci', 'imbgeco']

#var 'stfeco' satisfaccion con la situacion economica del pais
mappingeco = {'Insatisfecho': 0, 'SePuedeMejorar': 1, 'Satisfecho': 2}
def demo(valor):
    if 0 <= valor <= 3:
        return 'Insatisfecho'
    elif 4 <= valor <= 7:
        return 'SePuedeMejorar'
    else: 
        return 'Satisfecho'

dfc['stfeco'] = dfc['stfeco'].apply(demo)
dfc['stfeco_fc'] = dfc['stfeco'].map(mappingeco)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/stfeco_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappingeco, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['stfeco', 'stfeco_fc']].head())

In [ ]:
#BLOQUE 2: df_rd['actrolga','bctprd','dclagr','dclaid','dclcrm','dclenv', 'dclmig', 'dclwlfr', 
# 'dmcntov', 'euftf', 'gincdif', 'ginveco', 'lawobey', 'lrscale', 'polintr','stfeco','stfedu',
# 'stfgov', 'stfhlth', 'stflife', 'trstplc', 'trstplt', 'trstun', 'trstsci', 'imbgeco']
#Var  'stfedu'
mappingedu = {'Insatisfecho': 0, 'SePuedeMejorar': 1, 'Satisfecho': 2}
def demo(valor):
    if 0 <= valor <= 3:
        return 'Insatisfecho'
    elif 4 <= valor <= 7:
        return 'SePuedeMejorar'
    else: 
        return 'Satisfecho'

dfc['stfedu'] = dfc['stfedu'].apply(demo)
dfc['stfedu_fc'] = dfc['stfedu'].map(mappingedu)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/stfedu_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappingedu, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['stfedu', 'stfedu_fc']].head())

In [ ]:
#BLOQUE 2: df_rd['actrolga','bctprd','dclagr','dclaid','dclcrm','dclenv', 'dclmig', 'dclwlfr', 
# 'dmcntov', 'euftf', 'gincdif', 'ginveco', 'lawobey', 'lrscale', 'polintr','stfeco','stfedu',
# 'stfgov', 'stfhlth', 'stflife', 'trstplc', 'trstplt', 'trstun', 'trstsci', 'imbgeco']
#Var 'stfgov' Cuan satisfecho con el govierno nacional.
mappinggov = {'Insatisfecho': 0, 'SePuedeMejorar': 1, 'Satisfecho': 2}
def demo(valor):
    if 0 <= valor <= 3:
        return 'Insatisfecho'
    elif 4 <= valor <= 7:
        return 'SePuedeMejorar'
    else: 
        return 'Satisfecho'

dfc['stfgov'] = dfc['stfgov'].apply(demo)
dfc['stfgov_fc'] = dfc['stfgov'].map(mappinggov)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/stfgov_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappinggov, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['stfgov', 'stfgov_fc']].head())

In [ ]:
#BLOQUE 2: df_rd['actrolga','bctprd','dclagr','dclaid','dclcrm','dclenv', 'dclmig', 'dclwlfr', 
# 'dmcntov', 'euftf', 'gincdif', 'ginveco', 'lawobey', 'lrscale', 'polintr','stfeco','stfedu',
# 'stfgov', 'stfhlth', 'stflife', 'trstplc', 'trstplt', 'trstun', 'trstsci', 'imbgeco']
# Var 'stfhlth' cuan satisfecho con la sanidad publica.
mappinghth = {'Insatisfecho': 0, 'SePuedeMejorar': 1, 'Satisfecho': 2}
def demo(valor):
    if 0 <= valor <= 3:
        return 'Insatisfecho'
    elif 4 <= valor <= 7:
        return 'SePuedeMejorar'
    else: 
        return 'Satisfecho'

dfc['stfhlth'] = dfc['stfhlth'].apply(demo)
dfc['stfhlth_fc'] = dfc['stfhlth'].map(mappinghth)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/stfhlth_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappinghth, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['stfhlth', 'stfhlth_fc']].head())

In [ ]:
#BLOQUE 2: df_rd['actrolga','bctprd','dclagr','dclaid','dclcrm','dclenv', 'dclmig', 'dclwlfr', 
# 'dmcntov', 'euftf', 'gincdif', 'ginveco', 'lawobey', 'lrscale', 'polintr','stfeco','stfedu',
# 'stfgov', 'stfhlth', 'stflife', 'trstplc', 'trstplt', 'trstun', 'trstsci', 'imbgeco']
#Var stflife - Cuan satisfecho de la calidad de vida.
mappinglife = {'Insatisfecho': 0, 'SePuedeMejorar': 1, 'Satisfecho': 2}
def demo(valor):
    if 0 <= valor <= 3:
        return 'Insatisfecho'
    elif 4 <= valor <= 7:
        return 'SePuedeMejorar'
    else: 
        return 'Satisfecho'

dfc['stflife'] = dfc['stflife'].apply(demo)
dfc['stflife_fc'] = dfc['stflife'].map(mappinglife)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/stflife_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappinglife, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['stflife', 'stflife_fc']].head())

In [ ]:
#BLOQUE 2: df_rd['actrolga','bctprd','dclagr','dclaid','dclcrm','dclenv', 'dclmig', 'dclwlfr', 
# 'dmcntov', 'euftf', 'gincdif', 'ginveco', 'lawobey', 'lrscale', 'polintr','stfeco','stfedu',
# 'stfgov', 'stfhlth', 'stflife', 'trstplc', 'trstplt', 'trstun', 'trstsci', 'imbgeco']
#Var trstplc - Cuanta confianza en la policia
mappingplc = {'Desconfiado': 0, 'Esceptico': 1, 'Confiado': 2}
def demo(valor):
    if 0 <= valor <= 3:
        return 'Desconfiado'
    elif 4 <= valor <= 7:
        return 'Esceptico'
    else: 
        return 'Confiado'

dfc['trstplc'] = dfc['trstplc'].apply(demo)
dfc['trstplc_fc'] = dfc['trstplc'].map(mappingplc)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/trstplc_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappingplc, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['trstplc', 'trstplc_fc']].head())

In [ ]:
#BLOQUE 2: df_rd['actrolga','bctprd','dclagr','dclaid','dclcrm','dclenv', 'dclmig', 'dclwlfr', 
# 'dmcntov', 'euftf', 'gincdif', 'ginveco', 'lawobey', 'lrscale', 'polintr','stfeco','stfedu',
# 'stfgov', 'stfhlth', 'stflife', 'trstplc', 'trstplt', 'trstun', 'trstsci', 'imbgeco']
#Var trstplt  - Cuanta confianza en los politicos
mappingplc = {'Desconfiado': 0, 'Esceptico': 1, 'Confiado': 2}
def demo(valor):
    if 0 <= valor <= 3:
        return 'Desconfiado'
    elif 4 <= valor <= 7:
        return 'Esceptico'
    else: 
        return 'Confiado'

dfc['trstplt'] = dfc['trstplt'].apply(demo)
dfc['trstplt_fc'] = dfc['trstplt'].map(mappingplc)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/trstplt_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappingplc, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['trstplt', 'trstplt_fc']].head())

In [ ]:
#BLOQUE 2: df_rd['actrolga','bctprd','dclagr','dclaid','dclcrm','dclenv', 'dclmig', 'dclwlfr', 
# 'dmcntov', 'euftf', 'gincdif', 'ginveco', 'lawobey', 'lrscale', 'polintr','stfeco','stfedu',
# 'stfgov', 'stfhlth', 'stflife', 'trstplc', 'trstplt', 'trstun', 'trstsci', 'imbgeco']
#Var trstun  - Cuanta confianza en laa organizacion Naciones Unidas.
mappingplc = {'Desconfiado': 0, 'Esceptico': 1, 'Confiado': 2}
def demo(valor):
    if 0 <= valor <= 3:
        return 'Desconfiado'
    elif 4 <= valor <= 7:
        return 'Esceptico'
    else: 
        return 'Confiado'

dfc['trstun'] = dfc['trstun'].apply(demo)
dfc['trstun_fc'] = dfc['trstun'].map(mappingplc)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/trstun_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappingplc, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['trstun', 'trstun_fc']].head())

In [ ]:
#BLOQUE 2: df_rd['actrolga','bctprd','dclagr','dclaid','dclcrm','dclenv', 'dclmig', 'dclwlfr', 
# 'dmcntov', 'euftf', 'gincdif', 'ginveco', 'lawobey', 'lrscale', 'polintr','stfeco','stfedu',
# 'stfgov', 'stfhlth', 'stflife', 'trstplc', 'trstplt', 'trstun', 'trstsci', 'imbgeco']
#Var trstsci  - Cuanta confianza en el lobby cientifico
mappingplc = {'Desconfiado': 0, 'Esceptico': 1, 'Confiado': 2}
def demo(valor):
    if 0 <= valor <= 3:
        return 'Desconfiado'
    elif 4 <= valor <= 7:
        return 'Esceptico'
    else: 
        return 'Confiado'

dfc['trstsci'] = dfc['trstsci'].apply(demo)
dfc['trstsci_fc'] = dfc['trstsci'].map(mappingplc)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/trstsci_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappingplc, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['trstsci', 'trstsci_fc']].head())

In [ ]:
#BLOQUE 2: df_rd['actrolga','bctprd','dclagr','dclaid','dclcrm','dclenv', 'dclmig', 'dclwlfr', 
# 'dmcntov', 'euftf', 'gincdif', 'ginveco', 'lawobey', 'lrscale', 'polintr','stfeco','stfedu',
# 'stfgov', 'stfhlth', 'stflife', 'trstplc', 'trstplt', 'trstun', 'trstsci', 'imbgeco']
#Var imbgeco - Cuanto consideras que la inmigracion es buena para la economia.
mappinginm = {'InfluyeNegativamente': 0, 'Indiferente': 1, 'InfluyePositivamente': 2}
def demo(valor):
    if 0 <= valor <= 3:
        return 'InfluyeNegativamente'
    elif 4 <= valor <= 7:
        return 'Indiferente'
    else: 
        return 'InfluyePositivamente'

dfc['imbgeco'] = dfc['imbgeco'].apply(demo)
dfc['imbgeco_fc'] = dfc['imbgeco'].map(mappinginm )  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/imbgeco_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappinginm , f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['imbgeco', 'imbgeco_fc']].head())

In [38]:
#Eliminamos las columnas no factorizadas del bloque 2.
dfc = dfc.drop(columns=['imbgeco', 'cntry', 'trstsci', 'trstun', 'trstplt', 'trstplc', 
                        'stflife', 'stfhlth', 'stfgov', 'stfedu', 'stfeco', 'euftf', 'dmcntov'])

In [ ]:
#Factorizaremos las escalas de 0 a 10 del bloque 3.
# #Bloque 3: df_rd['aesfdrk','atchctr','atcherp','blgetmg','brncntr','dscrgnd','dscrrlg',
# 'facntr','happy','health','inprdsc','mocntr','rlgdgr','ccnthum','sclmeet','vteumbgb']
#Var atchctr - Cuan patriota eres? [country]
mappingemo = {'Desapegado': 0, 'ConCiertoApego': 1, 'MuyApegado': 2}
def demo(valor):
    if 0 <= valor <= 3:
        return 'Desapegado'
    elif 4 <= valor <= 7:
        return 'ConCiertoApego'
    else: 
        return 'MuyApegado'

dfc['atchctr'] = dfc['atchctr'].apply(demo)
dfc['atchctr_fc'] = dfc['atchctr'].map(mappingemo)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/atchctr_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappingemo , f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['atchctr', 'atchctr_fc']].head())


In [ ]:
#Var atcherp - How emotionally attached to Europe
mappingemo = {'Desapegado': 0, 'ConCiertoApego': 1, 'MuyApegado': 2}
def demo(valor):
    if 0 <= valor <= 3:
        return 'Desapegado'
    elif 4 <= valor <= 7:
        return 'ConCiertoApego'
    else: 
        return 'MuyApegado'

dfc['atcherp'] = dfc['atcherp'].apply(demo)
dfc['atcherp_fc'] = dfc['atcherp'].map(mappingemo)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/atcherp_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappingemo , f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['atcherp', 'atcherp_fc']].head())

In [ ]:
#Var happy - How happy are you
mappinghppy = {'infeliz': 0, 'FelizPorMomentos': 1, 'Feliz': 2}

def demo(valor):
    if 0 <= valor <= 3:
        return 'infeliz'
    elif 4 <= valor <= 7:
        return 'FelizPorMomentos'
    else: 
        return 'Feliz'

dfc['happy'] = dfc['happy'].apply(demo)
dfc['happy_fc'] = dfc['happy'].map(mappinghppy )  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/happy_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappinghppy, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['happy', 'happy_fc']].head())

In [ ]:
#Var rlgdgr
mappingrel = {'Ateo': 0, 'ConCiertaCreencia': 1, 'Agnostico/Practicante': 2}

def demo(valor):
    if 0 <= valor <= 3:
        return 'Ateo'
    elif 4 <= valor <= 7:
        return 'ConCiertaCreencia'
    else: 
        return 'Agnostico/Practicante'

dfc['rlgdgr'] = dfc['rlgdgr'].apply(demo)
dfc['rlgdgr_fc'] = dfc['rlgdgr'].map(mappingrel)  # Usamos mappingtrust aquí

# Guarda el mapeo en un archivo JSON (opcional, pero útil para referencia futura)
ruta_archivo_json = "C:/GitHubRepos/ProyectoFinal/data/processed/rlgdgr_fc.json"
with open(ruta_archivo_json, 'w') as f:
    json.dump(mappingrel, f)  # Guardamos mappingtrust en el archivo JSON

print(dfc[['rlgdgr', 'rlgdgr_fc']].head())

In [43]:
dfc = dfc.drop(columns=['atchctr','atcherp','happy','rlgdgr'])

In [44]:
del dfc['ideo'], dfc['orig'],

In [ ]:
dfc.to_csv('C:\GitHubRepos\ProyectoFinal\data\processed\dfc.csv', index=False)

Bloque 4 no tiene escalas 0 10. 
Realizaremos un tanteo con el nuevo dataset, teniendo la posibilidad
de escoger la variable target en escala 0 10 o factorizada y comparamos scores.

In [46]:
#Eliminamos todas las variables del notebook para optimizar recursos
%reset -f

In [ ]:
import pandas as pd
import numpy as np, random
from sklearn.model_selection import train_test_split
random.seed(42)
dtest = pd.read_csv('C:\GitHubRepos\ProyectoFinal\data\processed\dfc.csv')

for col in dtest.select_dtypes(include=['float64']).columns:
    
    try:
        dtest[col] = dtest[col].astype('int64')
    except:
        dtest[col] = pd.to_numeric(dtest[col], downcast='integer')

print(dtest.dtypes)

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
import os

# 1. Carga de datos
dtest = pd.read_csv("C:\GitHubRepos\ProyectoFinal\data\processed\dfc.csv")

# 2. División train/test inicial (CON MUESTREO ESTRATIFICADO GENERAL)
X = dtest.drop(['ideo_fc', 'lrscale', 'cntryfcz', 'origfcz', 'cluster'], axis=1)
y = dtest['ideo_fc']

X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
    X, y, test_size=0.3,  random_state=42
)#stratify=y,

#Muestreo estratificado general
sampled_data_general = []
n_samples_general = min(4000, len(X_train_full))
X_train_general = X_train_full.sample(n=n_samples_general, random_state=42)
y_train_general = y_train_full.loc[X_train_general.index]

X_train_general, X_test_general, y_train_general, y_test_general = train_test_split(
    X_train_general, y_train_general, test_size=0.3, random_state=42
)#stratify=y_train_general

# 3. Guardado en CSV (opcional)
X_train_general.to_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/X_train_general.csv', index=False)
X_test_general.to_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/X_test_general.csv', index=False)
pd.DataFrame(y_train_general).to_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/y_train_general.csv', index=False)
pd.DataFrame(y_test_general).to_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/y_test_general.csv', index=False)


# 4. Iteración por clusters (CON MUESTREO ESTRATIFICADO POR CLUSTER)
for cluster in dtest['cluster'].unique():
    # Filtrar *X_train_full e y_train_full* por cluster (USANDO X_train_full)
    X_train_cluster = X_train_full[dtest.loc[X_train_full.index, 'cluster'] == cluster]
    y_train_cluster = y_train_full[dtest.loc[X_train_full.index, 'cluster'] == cluster]

    # Filtrar *X_test_full e y_test_full* por cluster (USANDO X_test_full)
    X_test_cluster = X_test_full[dtest.loc[X_test_full.index, 'cluster'] == cluster]
    y_test_cluster = y_test_full[dtest.loc[X_test_full.index, 'cluster'] == cluster]

    #Muestreo estratificado por cluster
    sampled_data_cluster_train = []
    n_samples_cluster_train = min(700, len(X_train_cluster)) #375 train + 125 test = 500
    X_train_cluster_sampled = X_train_cluster.sample(n=n_samples_cluster_train, random_state=42)
    y_train_cluster_sampled = y_train_cluster.loc[X_train_cluster_sampled.index]

    sampled_data_cluster_test = []
    n_samples_cluster_test = min(300, len(X_test_cluster)) #375 train + 125 test = 500
    X_test_cluster_sampled = X_test_cluster.sample(n=n_samples_cluster_test, random_state=42)
    y_test_cluster_sampled = y_test_cluster.loc[X_test_cluster_sampled.index]

    # 5. Submuestreo y Sobremuestreo (SMOTE después de RUS)
    rus = RandomUnderSampler(random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train_cluster_sampled, y_train_cluster_sampled)

    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_resampled, y_train_resampled)  # ¡CORREGIDO!

    # 7. Creación de DataFrames para cada cluster
    train_cluster_df = pd.DataFrame(X_train_resampled, columns=X_train_cluster.columns)
    train_cluster_df['ideo_fc'] = y_train_resampled

    test_cluster_df = pd.DataFrame(X_test_cluster_sampled, columns = X_test_cluster.columns) #Añadido los nombres de las columnas
    test_cluster_df['ideo_fc'] = y_test_cluster_sampled

    # 8. Guardado en CSVs para cada cluster (SOBREESCRIBIR)
    train_cluster_df.to_csv(
        f'C:/GitHubRepos/ProyectoFinal/data/models/Split/train52C{cluster}.csv', index=False
    )
    test_cluster_df.to_csv(
        f'C:/GitHubRepos/ProyectoFinal/data/models/Split/test52C{cluster}.csv', index=False
    )

    print(f"Archivos CSV para cluster {cluster} creados/sobreescritos exitosamente.")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2
from xgboost import XGBClassifier  # Import XGBoost
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import joblib
import json
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder

# --- Función principal ---
def entrenar_y_evaluar(X_train, y_train, X_test, y_test, target_variable, model_type, ruta_modelo, ruta_configuracion, ruta_dataset):
    resultados = {}
    mejor_modelo = None
    mejor_k = None
    mejores_resultados = None
    mejor_auc = -1  # Inicializamos con el peor valor posible
    mejor_target_variable = None

    for k in range(9, 39):  # Range of k values to test
        # Selección de características
        selector = SelectKBest(score_func=chi2, k=k)
        X_train_new = selector.fit_transform(X_train, y_train)
        X_test_new = selector.transform(X_test)
        mejores_features = X_train.columns[selector.get_support()]

        # --- Tratamiento de clases minoritarias con SMOTE ---
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_new, y_train)

        # Optimización de hiperparámetros con RandomizedSearchCV (XGBoost)

        # 1. Label Encode your target variable *before* using it in XGBoost
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)  # Fit and transform on training data
        y_test_encoded = le.transform(y_test)  # Transform the test using the fitted encoder

        model = XGBClassifier(random_state=42, eval_metric='logloss')  # XGBoost - use_label_encoder removed
        param_grid = {
            'n_estimators': range(50, 301, 50),  # Wider range
            'max_depth': range(3, 11),  # Range for max_depth
            'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Learning rate
            'gamma': [0, 0.1, 0.2],  # Gamma for regularization
            'min_child_weight': range(1, 6),  # Min child weight
            'subsample': [0.6, 0.8, 1.0],  # Subsample ratio
            'colsample_bytree': [0.6, 0.8, 1.0],  # Colsample by tree
            'reg_alpha': [0, 0.1, 1],  # L1 regularization
            'reg_lambda': [0, 0.1, 1]  # L2 regularization
        }
        scoring = 'f1_weighted'
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        X_train_opt = X_train_resampled
        y_train_opt = y_train_resampled

        random_search = RandomizedSearchCV(model, param_grid, n_iter=50, scoring=scoring, cv=cv, n_jobs=-1, random_state=42)  # Increased n_iter
        random_search.fit(X_train_opt, y_train_opt)
        best_model = random_search.best_estimator_

        # Evaluación y almacenamiento de resultados
        y_pred = best_model.predict(X_test_new)  # Predictions will be numerical
        y_pred_decoded = le.inverse_transform(y_pred)  # Decode to original labels
        accuracy = accuracy_score(y_test, y_pred_decoded)  # Compare with original labels
        precision = precision_score(y_test, y_pred_decoded, average='weighted')
        recall = recall_score(y_test, y_pred_decoded, average='weighted')
        f1 = f1_score(y_test, y_pred_decoded, average='weighted')
        auc = roc_auc_score(y_test, best_model.predict_proba(X_test_new), multi_class='ovr')

        resultados[(target_variable, k)] = {
            'mejores_features': mejores_features.tolist(),
            'best_params': random_search.best_params_,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': auc
        }

        print(f"Resultados para {target_variable}, k={k}:")
        print(f"Mejores características: {mejores_features}")
        print(f"Mejores hiperparámetros: {random_search.best_params_}")
        print(f"Exactitud: {accuracy}")
        print(f"Precisión: {precision}")
        print(f"Recall: {recall}")
        print(f"F1-score: {f1}")
        print(f"AUC: {auc}")
        print("-" * 30)

        if auc > mejor_auc:
            mejor_auc = auc
            mejor_modelo = best_model
            mejor_k = k
            mejores_resultados = resultados[(target_variable, k)]
            mejor_target_variable = target_variable

    # Entrenar el modelo final con la mejor configuración (usando TODO el conjunto de entrenamiento)
    selector_final = SelectKBest(score_func=chi2, k=mejor_k)
    X_final = selector_final.fit_transform(X_train, y_train)  # Use X_train and y_train
    rf_final = XGBClassifier(random_state=42, eval_metric='logloss', **mejores_resultados['best_params'])
    rf_final.fit(X_final, le.transform(y_train))  # Encode y_train here as well

    # Guardar el modelo (el MEJOR modelo)
    joblib.dump(rf_final, ruta_modelo)

    # Guardar la configuración (la MEJOR configuración)
    with open(ruta_configuracion, 'w') as f:
        json.dump({'k': mejor_k, 'resultados': mejores_resultados, 'target_variable': mejor_target_variable, 'best_params': mejores_resultados['best_params']}, f)

    # Guardar el dataset final
    df_final = pd.DataFrame(X_final, columns=X_train.columns[selector_final.get_support()])
    df_final[mejor_target_variable] = y_train.loc[df_final.index] #Use y_train
    df_final.to_csv(ruta_dataset, index=False)

    print(f"\nMejor modelo guardado en: {ruta_modelo}")
    print(f"Configuración guardada en: {ruta_configuracion}")
    print(f"Dataset guardado en: {ruta_dataset}")
    
pretrain = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data\models\Split/test52C2.csv',index_col=False)
pretest = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/train52C2.csv',index_col=False)
X_train = pretrain.drop('ideo_fc', axis=1)
y_train = pretrain['ideo_fc'].squeeze()
X_test = pretest.drop('ideo_fc', axis=1)
y_test = pretest['ideo_fc'].squeeze()
'''
X_train = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/X_train_general.csv')
y_train = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/y_train_general.csv')
X_test = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/X_test_general.csv')
y_test = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/y_test_general.csv')
y_train = y_train.squeeze()  # Convierte y_train a Serie
y_test = y_test.squeeze() #Convierte y_test a serie'''
ruta_modelo= 'C:/GitHubRepos/ProyectoFinal/data/models/BRF_ideo_fc.pkl'
ruta_configuracion= 'C:/GitHubRepos/ProyectoFinal/data/models/BRFsetup_ideo_fc.json'
ruta_dataset = 'C:/GitHubRepos/ProyectoFinal/data/models/Datasets/dfRF_ideo_fc.csv'
target_variable = 'ideo_fc'
model_type = RandomForestClassifier
entrenar_y_evaluar(X_train, y_train, X_test, y_test, target_variable, model_type, ruta_modelo, ruta_configuracion, ruta_dataset)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
import joblib
import json
from imblearn.over_sampling import SMOTE

# --- Función principal para Regresión Logística ---
def entrenar_y_evaluar_regresion_logistica(X_train, y_train, X_test, y_test, target_variable, ruta_modelo, ruta_configuracion, ruta_dataset):
    resultados = {}
    mejor_modelo = None
    mejor_k = None
    mejores_resultados = None
    mejor_auc = -1

    for k in range(9, 39):  # Rango de valores de k a probar
        # Selección de características
        selector = SelectKBest(score_func=chi2, k=k) #o f_classif si hay negativos
        X_train_new = selector.fit_transform(X_train, y_train)
        X_test_new = selector.transform(X_test)
        mejores_features = X_train.columns[selector.get_support()]

        # --- Tratamiento de clases minoritarias con SMOTE ---
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_new, y_train)

        # Escalado de características (¡MUY IMPORTANTE para Regresión Logística!)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_resampled)  # Ajustar y transformar en entrenamiento
        X_test_scaled = scaler.transform(X_test_new)  # Transformar en prueba (usando el scaler ajustado en entrenamiento)


        # Optimización de hiperparámetros con RandomizedSearchCV
        model = LogisticRegression(random_state=42, solver='liblinear', max_iter=10000)  # solver 'liblinear' para penalización L1 y L2
        param_grid = {
            'C': np.logspace(-4, 4, 20),  # Valores de C para regularización
            'penalty': ['l1', 'l2'],  # Penalización L1 o L2
        }
        scoring = 'f1_weighted' #Métrica a optimizar
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        random_search = RandomizedSearchCV(model, param_grid, n_iter=20, scoring=scoring, cv=cv, n_jobs=-1, random_state=42)
        random_search.fit(X_train_scaled, y_train_resampled)
        best_model = random_search.best_estimator_

        # Evaluación y almacenamiento de resultados
        y_pred = best_model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        try: #Manejo de error en caso de que la variable objetivo tenga solo una clase
            auc = roc_auc_score(y_test, best_model.predict_proba(X_test_scaled), multi_class='ovr')
        except ValueError:
            auc = 0

        resultados[(target_variable, k)] = {
            'mejores_features': mejores_features.tolist(),
            'best_params': random_search.best_params_,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': auc
        }

        print(f"Resultados para {target_variable}, k={k}:")
        print(f"Mejores características: {mejores_features}")
        print(f"Mejores hiperparámetros: {random_search.best_params_}")
        print(f"Exactitud: {accuracy}")
        print(f"Precisión: {precision}")
        print(f"Recall: {recall}")
        print(f"F1-score: {f1}")
        print(f"AUC: {auc}")
        print("-" * 30)

        if auc > mejor_auc:
            mejor_auc = auc
            mejor_modelo = best_model
            mejor_k = k
            mejores_resultados = resultados[(target_variable, k)]
            mejor_target_variable = target_variable

    # Entrenar el modelo final con la mejor configuración (usando TODO el conjunto de entrenamiento)
    selector_final = SelectKBest(score_func=chi2, k=mejor_k) #o f_classif si hay negativos
    X_final = selector_final.fit_transform(X_train, y_train)
    scaler_final = StandardScaler()
    X_final_scaled = scaler_final.fit_transform(X_final)
    lr_final = LogisticRegression(solver='liblinear', max_iter=10000, **mejores_resultados['best_params'])
    lr_final.fit(X_final_scaled, y_train)

    # Guardar el modelo (el MEJOR modelo)
    joblib.dump(lr_final, ruta_modelo)

    # Guardar la configuración (la MEJOR configuración)
    with open(ruta_configuracion, 'w') as f:
        json.dump({'k': mejor_k, 'resultados': mejores_resultados, 'target_variable': mejor_target_variable, 'best_params': mejores_resultados['best_params']}, f)

    # Guardar el dataset final
    df_final = pd.DataFrame(X_final, columns=X_train.columns[selector_final.get_support()])
    df_final[mejor_target_variable] = y_train.loc[df_final.index] #Use y_train
    df_final.to_csv(ruta_dataset, index=False)

    print(f"\nMejor modelo guardado en: {ruta_modelo}")
    print(f"Configuración guardada en: {ruta_configuracion}")
    print(f"Dataset guardado en: {ruta_dataset}")

# Ejemplo de uso (reemplaza con tus rutas y datos)
pretrain = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data\models\Split/test52C2.csv',index_col=False)
pretest = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/train52C2.csv',index_col=False)
X_train = pretrain.drop('ideo_fc', axis=1)
y_train = pretrain['ideo_fc'].squeeze()
X_test = pretest.drop('ideo_fc', axis=1)
y_test = pretest['ideo_fc'].squeeze()

ruta_modelo = 'C:/GitHubRepos/ProyectoFinal/data/models/LR_ideo_fc.pkl'
ruta_configuracion = 'C:/GitHubRepos/ProyectoFinal/data/models/LRsetup_ideo_fc.json'
ruta_dataset = 'C:/GitHubRepos/ProyectoFinal/data/models/Datasets/dfLR_ideo_fc.csv'
target_variable = 'ideo_fc'

entrenar_y_evaluar_regresion_logistica(X_train, y_train, X_test, y_test, target_variable, ruta_modelo, ruta_configuracion, ruta_dataset)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB  # Modelo Bayesiano Gaussiano
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
import joblib
import json
from imblearn.over_sampling import SMOTE

# --- Función principal para Naive Bayes Gaussiano ---
def entrenar_y_evaluar_naive_bayes(X_train, y_train, X_test, y_test, target_variable, ruta_modelo, ruta_configuracion, ruta_dataset):
    resultados = {}
    mejor_modelo = None
    mejor_k = None
    mejores_resultados = None
    mejor_auc = -1

    for k in range(9, 39):  # Rango de valores de k a probar
        # Selección de características
        selector = SelectKBest(score_func=chi2, k=k)  # o f_classif si hay negativos
        X_train_new = selector.fit_transform(X_train, y_train)
        X_test_new = selector.transform(X_test)
        mejores_features = X_train.columns[selector.get_support()]

        # --- Tratamiento de clases minoritarias con SMOTE ---
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_new, y_train)

        # Escalado de características (Opcional para Naive Bayes, pero a veces ayuda)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_resampled)
        X_test_scaled = scaler.transform(X_test_new)

        # Entrenamiento del modelo Naive Bayes Gaussiano
        model = GaussianNB()  # No hay hiperparámetros para ajustar en este caso
        model.fit(X_train_scaled, y_train_resampled)

        # Evaluación y almacenamiento de resultados
        y_pred = model.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        try:
            auc = roc_auc_score(y_test, model.predict_proba(X_test_scaled), multi_class='ovr')
        except ValueError:
            auc = 0

        resultados[(target_variable, k)] = {
            'mejores_features': mejores_features.tolist(),
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': auc
        }

        print(f"Resultados para {target_variable}, k={k}:")
        print(f"Mejores características: {mejores_features}")
        print(f"Exactitud: {accuracy}")
        print(f"Precisión: {precision}")
        print(f"Recall: {recall}")
        print(f"F1-score: {f1}")
        print(f"AUC: {auc}")
        print("-" * 30)

        if auc > mejor_auc:
            mejor_auc = auc
            mejor_modelo = model
            mejor_k = k
            mejores_resultados = resultados[(target_variable, k)]
            mejor_target_variable = target_variable

    # Entrenar el modelo final con la mejor configuración (usando TODO el conjunto de entrenamiento)
    selector_final = SelectKBest(score_func=chi2, k=mejor_k)  # o f_classif si hay negativos
    X_final = selector_final.fit_transform(X_train, y_train)
    scaler_final = StandardScaler()
    X_final_scaled = scaler_final.fit_transform(X_final)
    nb_final = GaussianNB()
    nb_final.fit(X_final_scaled, y_train)

    # Guardar el modelo (el MEJOR modelo)
    joblib.dump(nb_final, ruta_modelo)

    # Guardar la configuración (la MEJOR configuración)
    with open(ruta_configuracion, 'w') as f:
        json.dump({'k': mejor_k, 'resultados': mejores_resultados, 'target_variable': mejor_target_variable}, f)

    # Guardar el dataset final
    df_final = pd.DataFrame(X_final, columns=X_train.columns[selector_final.get_support()])
    df_final[mejor_target_variable] = y_train.loc[df_final.index]  # Use y_train
    df_final.to_csv(ruta_dataset, index=False)

    print(f"\nMejor modelo guardado en: {ruta_modelo}")
    print(f"Configuración guardada en: {ruta_configuracion}")
    print(f"Dataset guardado en: {ruta_dataset}")


# Ejemplo de uso (reemplaza con tus rutas y datos)
pretrain = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data\models\Split/test52C2.csv', index_col=False)
pretest = pd.read_csv('C:/GitHubRepos/ProyectoFinal/data/models/Split/train52C2.csv', index_col=False)
X_train = pretrain.drop('ideo_fc', axis=1)
y_train = pretrain['ideo_fc'].squeeze()
X_test = pretest.drop('ideo_fc', axis=1)
y_test = pretest['ideo_fc'].squeeze()

ruta_modelo = 'C:/GitHubRepos/ProyectoFinal/data/models/NB_ideo_fc.pkl'
ruta_configuracion = 'C:/GitHubRepos/ProyectoFinal/data/models/NBsetup_ideo_fc.json'
ruta_dataset = 'C:/GitHubRepos/ProyectoFinal/data/models/Datasets/dfNB_ideo_fc.csv'
target_variable = 'ideo_fc'

entrenar_y_evaluar_naive_bayes(X_train, y_train, X_test, y_test, target_variable, ruta_modelo, ruta_configuracion, ruta_dataset)

In [80]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib
import json
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# --- Función principal para Red Neuronal ---
def entrenar_y_evaluar_red_neuronal(X_train, y_train, X_test, y_test, target_variable, ruta_modelo, ruta_configuracion, ruta_dataset):
    resultados = {}
    mejor_modelo = None
    mejor_k = None
    mejores_resultados = None
    mejor_auc = -1

    for k in range(9, 39):  # Rango de valores de k a probar
        # Selección de características
        selector = SelectKBest(score_func=chi2, k=k)
        X_train_new = selector.fit_transform(X_train, y_train)
        X_test_new = selector.transform(X_test)
        mejores_features = X_train.columns[selector.get_support()]

        # --- Tratamiento de clases minoritarias con SMOTE ---
        smote = SMOTE(random_state=42)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_new, y_train)

        # Escalado de características
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_resampled)
        X_test_scaled = scaler.transform(X_test_new)

        # --- Diseño y compilación del modelo de red neuronal ---
        model = keras.Sequential([
            layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
            layers.Dropout(0.2),  # Capa de dropout para regularización
            layers.Dense(32, activation='relu'),
            layers.Dropout(0.2),
            layers.Dense(len(np.unique(y_train)), activation='softmax')  # Capa de salida con softmax
        ])

        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',  # o 'categorical_crossentropy' si y_train está en one-hot encoding
                      metrics=['accuracy'])

        # Entrenamiento del modelo
        model.fit(X_train_scaled, y_train_resampled, epochs=50, batch_size=32, verbose=0)  # Ajusta epochs y batch_size

        # Evaluación y almacenamiento de resultados
        y_pred = np.argmax(model.predict(X_test_scaled), axis=1)  # Obtener las clases predichas
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        try:
            auc = roc_auc_score(y_test, model.predict(X_test_scaled), multi_class='ovr', average='weighted')
        except ValueError:
            auc = 0

        resultados[(target_variable, k)] = {
            'mejores_features': mejores_features.tolist(),
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': auc
        }

        print(f"Resultados para {target_variable}, k={k}:")
        print(f"Mejores características: {mejores_features}")
        print(f"Exactitud: {accuracy}")
        print(f"Precisión: {precision}")
        print(f"Recall: {recall}")
        print(f"F1-score: {f1}")
        print(f"AUC: {auc}")
        print("-" * 30)

        if auc > mejor_auc:
            mejor_auc = auc
            mejor_modelo = model
            mejor_k = k
            mejores_resultados = resultados[(target_variable, k)]
            mejor_target_variable = target_variable
            mejor_scaler = scaler
            mejor_selector = selector

    # Entrenar el modelo final con la mejor configuración (usando TODO el conjunto de entrenamiento)
    X_final = mejor_selector.transform(X_train)
    X_final_scaled = mejor_scaler.transform(X_final)
    nn_final = keras.models.clone_model(mejor_modelo)  # Crea un nuevo modelo con la misma arquitectura
    nn_final.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    nn_final.fit(X_final_scaled, y_train, epochs=50, batch_size=32, verbose=0)  # Reentrena con todos los datos

    # Guardar el modelo (el MEJOR modelo)
    nn_final.save(ruta_modelo)  # Guarda en formato h5

    # Guardar la configuración (la MEJOR configuración)
    with open(ruta_configuracion, 'w') as f:
        json.dump({'k': mejor_k, 'resultados': mejores_resultados, 'target_variable': mejor_target_variable}, f)

    # Guardar el dataset final
    df_final = pd.DataFrame(X_final, columns=X_train.columns[mejor_selector.get_support()])
    df_final[mejor_target_variable] = y_train.loc[df_final.index]  # Use y_train
    df_final.to_csv(ruta_dataset, index=False)

    print(f"\nMejor modelo guardado en: {ruta_modelo}")
    print(f"Configuración guardada en: {ruta_configuracion}")
    print(f"Dataset guardado en: {ruta_dataset}")

# Ejemplo de uso (reemplaza con tus rutas y datos)
# ... (tu código de carga de datos y definición de rutas)

entrenar_y_evaluar_red_neuronal(X_train, y_train, X_test, y_test, target_variable, ruta_modelo, ruta_configuracion, ruta_dataset)

ModuleNotFoundError: No module named 'tensorflow'